In [69]:
%matplotlib inline
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import rcParams
import seaborn
from tqdm import tqdm,tqdm_gui
from scipy.optimize import curve_fit
import sympy as sm
rcParams['figure.figsize'] = (12.0, 7.0)
import prettytable as pt

In [136]:
A = np.array([[10., -1., 2., 0.],
              [-1., 11., -1., 3.],
              [2., -1., 10., -1.],
              [0.0, 3., -1., 8.]])
b = np.array([6., 25., -11., 15.])
#Αρχικές τιμές X
x = np.zeros_like(b)
e=1e-3

In [114]:
def GEPP(A, b):
    '''
    Gaussian elimination with partial pivoting.
    % input: A is an n x n nonsingular matrix
    %        b is an n x 1 vector
    % output: x is the solution of Ax=b.
    % post-condition: A and b have been modified. 
    '''
    n =  len(A)
    if b.size != n:
        raise ValueError("Invalid argument: incompatible sizes between A & b.", b.size, n)
    # k represents the current pivot row. Since GE traverses the matrix in the upper 
    # right triangle, we also use k for indicating the k-th diagonal column index.
    for k in xrange(n-1):
        #Choose largest pivot element below (and including) k
        maxindex = np.abs(A[k:,k]).argmax() + k
        if A[maxindex, k] == 0:
            raise ValueError("Matrix is singular.")
        #Swap rows
        if maxindex != k:
            A[[k,maxindex]] = A[[maxindex, k]]
            b[[k,maxindex]] = b[[maxindex, k]]
        for row in xrange(k+1, n):
            multiplier = A[row][k]/A[k][k]
            #the only one in this column since the rest are zero
            A[row][k] = multiplier
            for col in xrange(k + 1, n):
                A[row][col] = A[row][col] - multiplier*A[k][col]
            #Equation solution column
            b[row] = b[row] - multiplier*b[k]
    print A
    print b
    x = np.zeros(n)
    k = n-1
    x[k] = b[k]/A[k,k]
    while k >= 0:
        x[k] = (b[k] - np.dot(A[k,k+1:],x[k+1:]))/A[k,k]
        k = k-1
    return x

In [135]:
GEPP(A,b)

[[  1.00000000e+01  -1.00000000e+00   2.00000000e+00   0.00000000e+00]
 [ -1.00000000e-19   1.08888889e+01  -7.77777778e-01   3.00000000e+00]
 [  2.00000000e-19   1.60571472e-19   9.49280465e+00  -7.64294003e-01]
 [  0.00000000e+00   5.94211875e-20   1.49831274e-19   7.02168421e+00]]
[  6.          25.66666667 -10.32166341   6.32358481]


array([ 1.006615  ,  2.03653787, -1.01480608,  0.90057949])

In [96]:
for n in range(10):
    temp=np.copy(x)
    for i in range(A.shape[0]):
        s=0.
        for j in range(A.shape[1]):
            if i<>j:
                s=s+A[i,j]*x[j]            
        x[i]=(b[i]-s)/A[i,i]
    #if np.all(np.abs(temp-x)<e):
    #    break
    print x

[ 1.00000836  2.00000117 -1.00000275  0.99999922]
[ 1.00000067  2.00000002 -1.00000021  0.99999996]
[ 1.00000004  1.99999999 -1.00000001  1.        ]
[ 1.  2. -1.  1.]
[ 1.  2. -1.  1.]
[ 1.  2. -1.  1.]
[ 1.  2. -1.  1.]
[ 1.  2. -1.  1.]
[ 1.  2. -1.  1.]
[ 1.  2. -1.  1.]


In [30]:
def f(x): return x**2 *np.exp(x**3)-np.sin(x)

In [88]:
def df2(f,x,h): return (f(x+h)-f(x))/h
def df3(f,x,h): return (-f(x-h)+f(x+h))/(2.*h)
def df5(f,x,h): return (f(x-2.*h)-8.*f(x-h)+8.*f(x+h)-f(x+2.*h))/(12.*h)
def ddf2(f,x,h): return (f(x+2.*h)-2.*f(x+h)+f(x))/h**2
def ddf3(f,x,h): return (f(x-h)-2.*f(x)+f(x+h))/h**2
def ddf5(f,x,h): return (-f(x+2.*h)+16.*f(x+h)-30.*f(x)+16.*f(x-h)-f(x-2.*h))

In [89]:
x0=2.19
d1=2.67435e6
d2=4.3145e7
tt1=pt.PrettyTable(['h','dx 2Point (error)','dx 3Point (error)','dx 5Point (error)'])
tt2=pt.PrettyTable(['h','ddx 2Point (error)','ddx 3Point (error)','ddx 5Point (error)'])
for h in [1.,0.1,0.01]:
    tt1.add_row(['{}'.format(h),
               '{:5E} ({:.2E})'.format(df2(f,x0,h),np.abs(df2(f,x0,h)-d1)/d1),
               '{:5E} ({:.2E})'.format(df3(f,x0,h),np.abs(df3(f,x0,h)-d1)/d1),
               '{:5E} ({:.2E})'.format(df5(f,x0,h),np.abs(df5(f,x0,h)-d1)/d1)
              ])
    tt2.add_row(['{}'.format(h),
               '{:5E} ({:.2E})'.format(ddf2(f,x0,h),np.abs(ddf2(f,x0,h)-d2)/d2),
               '{:5E} ({:.2E})'.format(ddf3(f,x0,h),np.abs(ddf3(f,x0,h)-d2)/d2),
               '{:5E} ({:.2E})'.format(ddf5(f,x0,h),np.abs(ddf5(f,x0,h)-d2)/d2)
              ])
    #print '{} First Derivative 3Point {:5E} ({:.4f}),5Point {:5E} ({:.4f})'.format(h,df3(f,x0,h),np.abs(df3(f,x0,h)-d1)/d1,df5(f,x0,h),np.abs(df5(f,x0,h)-d1)/d1)
    #print '{} Second Derivative 3Point {:5E} ({:.4f}),5Point {:5E} ({:.4f})'.format(h,ddf3(f,x0,h),np.abs(ddf3(f,x0,h)-d2)/d2,ddf5(f,x0,h),np.abs(ddf3(f,x0,h)-d2)/d2)
print tt1
print tt2

+------+-------------------------+-------------------------+--------------------------+
|  h   |    dx 2Point (error)    |    dx 3Point (error)    |    dx 5Point (error)     |
+------+-------------------------+-------------------------+--------------------------+
| 1.0  | 1.275100E+15 (4.77E+08) | 6.375499E+14 (2.38E+08) | -1.294120E+32 (4.84E+25) |
| 0.1  | 6.864329E+06 (1.57E+00) | 4.104637E+06 (5.35E-01) | 1.439846E+06 (4.62E-01)  |
| 0.01 | 2.902786E+06 (8.54E-02) | 2.686523E+06 (4.55E-03) | 2.674268E+06 (3.05E-05)  |
+------+-------------------------+-------------------------+--------------------------+
+------+-------------------------+-------------------------+--------------------------+
|  h   |    ddx 2Point (error)   |    ddx 3Point (error)   |    ddx 5Point (error)    |
+------+-------------------------+-------------------------+--------------------------+
| 1.0  | 1.552944E+33 (3.60E+25) | 1.275100E+15 (2.96E+07) | -1.552944E+33 (3.60E+25) |
| 0.1  | 3.302438E+08 (6.65E+00)